#### Code to train classification models
**TODO**:
- verif dist fts
- GroupKFold ?
- inspect cat features
- catboost + xgboost + lgbm
- NaNs ?

In [50]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
cd ../src

/home/theo/kaggle/foursquare/src


## Imports

In [3]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = "1"

import torch
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 2080 Ti'

In [4]:
import os
import gc
import re
import glob
import json
import cudf
import lofo
import torch
import pickle
import optuna
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from copy import deepcopy
from tqdm.notebook import tqdm
from collections import Counter
from pandarallel import pandarallel
from numerize.numerize import numerize
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GroupKFold
from cuml.feature_extraction.text import TfidfVectorizer

pandarallel.initialize(progress_bar=False, use_memory_fs=False)
pd.options.display.max_columns = 500
pd.options.display.max_rows = 200
warnings.simplefilter("ignore")

INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [5]:
from pp import *
from dtypes import *
from params import *
from utils.metrics import *
from inference.main import k_fold_inf
from data.preparation import reduce_mem_usage
from matching import get_CV, load_cleaned_data, cci

### Params

In [51]:
N_FOLDS = 5
THRESHOLD = 0.005

## Data


In [52]:
df = pd.read_csv(DATA_PATH + "train.csv")[["id", "point_of_interest"]]

In [53]:
LEVEL = 2

In [54]:
# df_p = pd.read_csv(OUT_PATH + f"features_train_{LEVEL}.csv")

df_p = pd.read_csv(OUT_PATH + f"features_train_{LEVEL}_{THRESHOLD}.csv", dtype=DTYPES_2)

In [55]:
df_p = df_p.sort_values(['id_1', 'id_2']).reset_index(drop=True)

## Exp

In [77]:
# EXP_FOLDER = LOG_PATH + "lvl_2/" + "2022-07-03/0/"  # xgb 10kf 0.005

EXP_FOLDER = LOG_PATH + "lvl_2/" + "2022-07-01/0/"  # xgb 10kf 0.0075
EXP_FOLDER = LOG_PATH + "lvl_2/" + "2022-07-01/1/"  # lgb 10kf 0.0075

# EXP_FOLDER = LOG_PATH + "lvl_2/" + "2022-06-30/5/"  # lgb gkf - 0.8837
# EXP_FOLDER = LOG_PATH + "lvl_2/" + "2022-06-30/6/"  # xgb gkf - 0.8782

# EXP_FOLDERS = [
#     LOG_PATH + "lvl_2/" + "2022-06-30/5/",  # lgb gkf - 0.8837
#     LOG_PATH + "lvl_2/" + "2022-07-01/2/",   # xgb gkf - 0.8818
# ]

EXP_FOLDERS = [  # TH = 0.005 - kf
    LOG_PATH + "lvl_2/" + "2022-07-03/1/",  # lgb 10kf -
    LOG_PATH + "lvl_2/" + "2022-07-03/0/",  # xgb 10kf -
]

WEIGHTS = [0.9, 0.1]
# WEIGHTS = [1, 0]

In [78]:
# pred_oof = np.load(EXP_FOLDER + "pred_oof.npy")

pred_oof = np.average([
    np.load(f + "pred_oof.npy") for f in EXP_FOLDERS
], weights=WEIGHTS, axis=0)

In [79]:
# preds_lgbm = k_fold_inf(
#     df_p,
#     EXP_FOLDERS[0],
# )

# preds_xgb = k_fold_inf(
#     df_p,
#     EXP_FOLDERS[1],
# )

# pred_oof = WEIGHTS[0] * preds_lgbm + WEIGHTS[1] * preds_xgb

In [80]:
roc_auc_score(df_p['match'], pred_oof)

0.9863037296961601

In [71]:
roc_auc_score(df_p['match'], pred_oof)

0.9862609133196312

### Restrict to a subset comparable to LB

In [81]:
# SUBSET = [0, 1, 2]

# df = pd.read_csv(f'../output/folds_5.csv')
# df = df[df['fold'].isin(SUBSET)]

# pred_oof = pred_oof[(df_p['fold_1'].isin(SUBSET)) & (df_p['fold_2'].isin(SUBSET))]

# df_p = df_p[(df_p['fold_1'].isin(SUBSET)) & (df_p['fold_2'].isin(SUBSET))].reset_index(drop=True)

In [82]:
# roc_auc_score(df_p['match'], pred_oof)

### Add m_true column

In [83]:
if "m_true" not in df.columns:
    df = df.reset_index()
    df = df.sort_values(by=["point_of_interest", "id"]).reset_index(drop=True)

    id_all = np.array(df["id"])
    poi_all = np.array(df["point_of_interest"])
    poi0 = poi_all[0]
    id0 = id_all[0]

    di_poi = {}
    for i in range(1, df.shape[0]):
        if poi_all[i] == poi0:
            id0 = str(id0) + " " + str(id_all[i])
        else:
            di_poi[poi0] = str(id0) + " "  # need to have trailing space in m_true
            poi0 = poi_all[i]
            id0 = id_all[i]

    di_poi[poi0] = str(id0) + " "  # need to have trailing space in m_true
    df["m_true"] = df["point_of_interest"].map(di_poi)

    df = df.sort_values(by="index").reset_index(
        drop=True
    )  # sort back to original order
    df.drop("index", axis=1, inplace=True)

In [84]:
THRESHOLDS = [0.45, 0.6, 0.6, 0.9, 0.]  # FULL DATA

# THRESHOLDS = [0.3, 0.45, 0.45, 0.8, 0]  # RESTRICTED  - cuts = [0.37, 0.48, 0.43, 0.77]
# THRESHOLDS = [0.5, 0.5, 0.5, 0.8]

# THRESHOLDS = [0.37, 0.48, 0.43, 0.77, 0.]  # LB / RESTRICTED

threshold, threshold_small, threshold_big, threshold_merge_max, threshold_merge_avg = THRESHOLDS

In [85]:
id_to_poi, poi_to_id, poi_counts = get_improved_CV(df_p, pred_oof, df, THRESHOLDS)

CV 0.9034


In [76]:
id_to_poi, poi_to_id, poi_counts = get_improved_CV(df_p, pred_oof, df, THRESHOLDS)

CV 0.9034


In [67]:
id_to_poi, poi_to_id, poi_counts = get_improved_CV(df_p, pred_oof, df, THRESHOLDS)

CV 0.9033


In [49]:
id_to_poi, poi_to_id, poi_counts = get_improved_CV(df_p, pred_oof, df, THRESHOLDS)

CV 0.9028


In [40]:
id_to_poi, poi_to_id, poi_counts = get_improved_CV(df_p, pred_oof, df, THRESHOLDS)

CV 0.8991


ValueError: not enough values to unpack (expected 5, got 3)

In [24]:
id_to_poi, poi_to_id, preds = get_improved_CV(df_p, pred_oof, df, THRESHOLDS)

CV 0.8982


## PP

### Prepare

In [90]:
df2 = df_p.copy()
df2["match"] = np.copy(pred_oof) #.astype("float32")

try:
    df2 = df2[['id_1', 'id_2', "match"]]
    df2.columns = ['id', 'id2', "match"]
except KeyError:
    df2 = df2[['id', 'id2', "match"]]

# sort by decr prediction
df2 = df2.sort_values(by=["match"], ascending=False).reset_index(drop=True)

### Construct POI from pairs

In [91]:
id_to_poi, poi_to_id, poi_counts = match_pois(
    df2,
    threshold=threshold,
    threshold_small=threshold_small,
    threshold_big=threshold_big
)

### Merge POIs

#### Original

In [92]:
id_to_poi, poi_to_id, poi_counts = merge_pois_simple(
    df2,
    id_to_poi,
    poi_to_id,
    poi_counts,
    threshold=threshold,
    threshold_merge=threshold_merge_max,
)

#### Theo

In [93]:
# id_to_poi, poi_to_id, poi_counts = merge_pois_advanced(
#     df2,
#     id_to_poi,
#     poi_to_id,
#     poi_counts,
#     threshold_merge_avg=threshold_merge_avg,
#     threshold_merge_max=threshold_merge_max
# )

In [94]:
id_to_poi_svg = deepcopy(id_to_poi)

#### Remove outliers
- TODO : add lvl 1 as nomatch

In [98]:
id_to_poi = deepcopy(id_to_poi_svg)

poi_to_id = get_poi_to_id(id_to_poi)

In [100]:
# THRESHOLD_OUT = 0.3
THRESHOLD_OUT = 0.25  # sampled data

In [101]:
id_to_poi, poi_to_id = remove_outliers(df2, id_to_poi, poi_to_id, threshold_out=THRESHOLD_OUT)

In [102]:
# len(removed)  # TODO : retrieve matches for removed ??

In [103]:
# poi_to_id = {}
# for k in id_to_poi.keys():
#     if id_to_poi[k] not in poi_to_id:
#         poi_to_id[id_to_poi[k]] = []
#     poi_to_id[id_to_poi[k]].append(k)

### CV

In [104]:
preds = pd.DataFrame.from_dict(id_to_poi, orient="index").reset_index()
preds['matches'] = preds[0].map(poi_to_id).apply(lambda x: " ".join(x))
preds.columns = ["id", "poi", "m2"]

In [105]:
cv = evaluate(df, preds)
print(f"CV {cv:.5f}")

CV 0.90380


In [97]:
cv = evaluate(df, preds)
print(f"CV {cv:.5f}")

CV 0.90410


- base : 0.90228
- theo : 0.90312
- theo - outlier : 0.90405

- base : 0.9192
- theo : 0.91975
- theo - outlier : 0.9205